## Run the Following Cell to Load Libraries and Initialize Connection

In [ ]:
# pip install psycopg2-binary 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sqlalchemy import create_engine, text
import pickle
from pandas.api.types import is_numeric_dtype
import ipywidgets as widgets
from IPython.display import display
import psycopg2
dbname = 'gamecircus'
host = 'gc-dwh.cxykxux8ksim.us-east-1.redshift.amazonaws.com'
port = '5439'
conn = psycopg2.connect(dbname=dbname, user='dkhan', password='<PJR&w2(8gcE9j', host=host, port=port)

## Run the Following Cell to Choose Dates

In [ ]:
start_date_picker = widgets.DatePicker(description='Select Install Cohort Start Date', disabled=False)
end_date_picker = widgets.DatePicker(description='Select Install Cohort End Date', disabled=False)

display(start_date_picker,end_date_picker)

## Run the Following Cells

In [ ]:
start_date = start_date_picker.value.strftime('%Y-%m-%d')
end_date = end_date_picker.value.strftime('%Y-%m-%d')

In [ ]:
conn = psycopg2.connect(dbname=dbname, user='dkhan', password='<PJR&w2(8gcE9j', host=host, port=port)
script = f'''with main as (
	  select primary_id, media_source, campaign, sum(revenue) as total_revenue from revenue_user_summary
	  where install_date between '{start_date}' and '{end_date}'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  group by primary_id, media_source, campaign
	  ),

video as (
	  select primary_id, media_source, campaign, sum(revenue) as vid from revenue_user_summary
	  where install_date between '{start_date}' and '{end_date}'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  and revenue_type in ('rewarded_video')
	  group by primary_id, media_source, campaign
	  ),
	  
offerwall as (
	  select primary_id, media_source, campaign, sum(revenue) as ow from revenue_user_summary
	  where install_date between '{start_date}' and '{end_date}'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  and revenue_type in ('offerwall')
	  group by primary_id, media_source, campaign
	  ),
	  
	
interstitial as (
	  select primary_id, media_source, campaign, sum(revenue) as interstitial from revenue_user_summary
	  where install_date between '{start_date}' and '{end_date}'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  and revenue_type in ('interstitial')
	  group by primary_id, media_source, campaign
	  ),

banner as (
	  select primary_id, media_source, campaign, sum(revenue) as ban from revenue_user_summary
	  where install_date between '2023-11-08' and '2023-11-13'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  and revenue_type in ('banner')
	  group by primary_id, media_source, campaign
	  ),
	  	  
iap as (
	  select primary_id, media_source, campaign, sum(revenue) as iap from revenue_user_summary
	  where install_date between '{start_date}' and '{end_date}'
	  and app_name in ('Coin Dozer', 'Coin Dozer: Sweepstakes')
	  and revenue_source in ('IAP')
	  group by primary_id, media_source, campaign
	  ),
	  	  

  ddna_variant as (
  	select distinct userid, DDNAEVENTTRIGGEREDVARIANTNAME AS variant
  	from delta_dna_imports_21605_cdv_2023_11 dcd
  	where DDNAEVENTTRIGGEREDCAMPAIGNNAME Like 'QUICK_LV8_XP_2X (V2)' and gauserstartdate >= '2023-11-08'
  	)
  	
  	select main.primary_id, main.media_source, main.campaign, main.total_revenue, video.vid, offerwall.ow, interstitial.interstitial, banner.ban, iap.iap, ddna_variant.variant
  	from main left outer join video on main.primary_id = video.primary_id
  	left outer join offerwall on main.primary_id = offerwall.primary_id
  	left outer join interstitial on main.primary_id = interstitial.primary_id
  	left outer join banner on main.primary_id = banner.primary_id
  	left outer join iap on main.primary_id = iap.primary_id
  	left outer join ddna_variant on main.primary_id = ddna_variant.userid'''
    
output = pd.read_sql(script, con = conn)

In [ ]:
output.to_csv()

In [ ]:
path = input("Enter Output Path (without quotes)")

In [ ]:
output.to_csv(path + r'\Report.csv', index = False)